In [1]:
print("****THIS TARIFF CHANGER OUTPUTS SDIs THAT MATCH AGREEMENTS TARIFF****")

mpans = input("Please input the MPANs to run as csv format :: ")

****THIS TARIFF CHANGER OUTPUTS SDIs THAT MATCH AGREEMENTS TARIFF****


Please input the MPANs to run as csv format ::  '2000052732818', '1413096370000', '1900011373603', '1012710257249', '2200022578240', '2700008542920', '2200023066430', '1417489971007', '1012960517943', '1610000972221', '1610001861711', '2200020837687', '1412579302004', '2000010255246'


In [2]:
# SQL Query
print("^ Pull Data from Query")

import pandas as pd
import tentaclio as tio

kraken = 'databricks+thrift://hostname'
OUTPUT_PATH = '/Users/rob.cumpstey/Documents/Code'


configuration_query = f"""WITH max_record AS (
  SELECT meter_point_id, MAX(effective_from) AS effective_from
  FROM consumer.wh_electricity_estimated_annual_consumption_interval
  --WHERE meter_point_id = 4551126
  GROUP BY meter_point_id
),
eac_data AS (
  SELECT MP.mpan,
    EAC_TABLE.estimated_annual_consumption AS EAC,
    EAC_TABLE.time_pattern_regime_id AS tpr,
    EAC_TABLE.meter_point_id,
    EAC_TABLE.effective_from
  FROM consumer.wh_electricity_estimated_annual_consumption_interval AS EAC_TABLE
  INNER JOIN consumer.stg_properties_electricitymeterpoint MP ON EAC_TABLE.meter_point_id = MP.meter_point_id
  INNER JOIN max_record mr ON EAC_TABLE.meter_point_id = mr.meter_point_id AND mr.effective_from = EAC_TABLE.effective_from
  WHERE EAC_TABLE.is_currently_active_record = TRUE
),
total_eac AS (
  SELECT mpan,
    SUM(EAC) AS acc_eac
  FROM eac_data
  GROUP BY mpan
)
SELECT DISTINCT
  MPOINT.mpan AS MPAN,
  grid_supply_point_id AS GSP,
  METER.meter_type AS Type,
  METER.serial_number AS MSN,
  marketparticipant.market_participant_name AS MOP,
  wh.supplier_id AS SUPPLIER,
  te.acc_eac AS EAC,
  wh.tariff_code AS TARIFF,
  MAX(DATE(s2tariff.start_at)) AS agreement_start,
  DATE_ADD(MAX(DATE(s2tariff.start_at)), 1) AS EFD,
  st.latest_dc_name AS DC,
  MPOINT.supply_start_date AS SSD
FROM consumer.stg_properties_electricitymeterpoint AS MPOINT
LEFT JOIN consumer.stg_properties_electricitymeter AS METER ON METER.meter_point_id = MPOINT.meter_point_id
LEFT JOIN total_eac te ON MPOINT.mpan = te.mpan
LEFT JOIN consumer.stg_properties_electricityagentcontract AS AGENT ON AGENT.meter_point_id = MPOINT.meter_point_id
  AND AGENT.contract_type = 'MOP'
  AND AGENT.agent_contract_effective_from <= CURRENT_DATE
  AND COALESCE(AGENT.agent_contract_effective_to, '9999-01-01') > CURRENT_DATE
LEFT JOIN consumer.stg_electricity_marketparticipant AS marketparticipant ON marketparticipant.market_participant_id = AGENT.market_participant_id
LEFT JOIN consumer.wh_electricity_validated_register_configuration_agreement_rate_interval wh ON MPOINT.mpan = wh.mpan
LEFT JOIN consumer.wh_electricity_agreement_interval d ON wh.agreement_id = d.agreement_id
LEFT JOIN octoenergy_data_prod_prod.consumer.stg_smets2_tariff AS s2tariff ON s2tariff.mpxn = MPOINT.mpan
LEFT JOIN consumer.fnl_industry_settlement st ON MPOINT.mpan = st.mpan
WHERE MPOINT.mpan IN ({mpans})
  AND METER.active_to IS NULL
  AND wh.is_currently_active_agreement = TRUE
  AND wh.is_currently_active_register = TRUE
  AND d.is_currently_active_agreement = TRUE
GROUP BY MPOINT.mpan, grid_supply_point_id, METER.meter_type, METER.serial_number, marketparticipant.market_participant_name,
  wh.supplier_id, te.acc_eac, wh.tariff_code, st.latest_dc_name, MPOINT.supply_start_date
LIMIT 1000;"""

with tio.db(kraken) as query:
    configuration_query_results = query.get_df(configuration_query)
display(configuration_query_results)
# Export Data to CSV
configuration_query_results.to_csv('/home/jovyan/shared/Settlement/SSC Changer/variables.csv')

^ Pull Data from Query


,MPAN,GSP,Type,MSN,MOP,SUPPLIER,EAC,TARIFF,agreement_start,EFD,DC,SSD
0,1012710257249,1,S2A,21K0088514,LEYLAND METERING SERVICES LTD,MRCY,5172.9,E-2R-VAR-22-11-01-A,2024-02-14,2024-02-15,UDMS,2024-02-06
1,1413096370000,5,S1,17K0146492,LEYLAND METERING SERVICES LTD,MRCY,1755.0,E-2R-LOYAL-FIX-12M-24-02-08-E,2024-02-22,2024-02-23,UDMS,2022-03-05
2,1610000972221,7,S2ADE,22J0038426,LEYLAND METERING SERVICES LTD,MRCY,1014.0,E-2R-VAR-22-11-01-G,2024-02-20,2024-02-21,UDMS,2021-01-30
3,1412579302004,5,S2ADE,22L4610202,LEYLAND METERING SERVICES LTD,MRCY,2676.0,E-2R-VAR-22-11-01-E,2024-02-23,2024-02-24,UDMS,2021-11-04
4,1900011373603,9,S2AD,23E5126144,LEYLAND METERING SERVICES LTD,MRCY,6515.1,E-1R-VAR-22-11-01-J,2024-02-21,2024-02-22,UDMS,2023-10-06
5,2200020837687,11,S2ADE,20E5325334,OCTOPUS ENERGY SERVICES LTD,MRCY,2466.2,E-1R-VAR-22-11-01-L,2024-02-26,2024-02-27,UDMS,2020-04-19
6,2000010255246,8,S2ADE,21L4547276,LEYLAND METERING SERVICES LTD,MRCY,2087.3,E-2R-VAR-22-11-01-H,2024-02-23,2024-02-24,UDMS,2021-10-28
7,2200022578240,11,S2ADE,20E5408300,OCTOPUS ENERGY SERVICES LTD,MRCY,6264.1,E-2R-LOYAL-FIX-12M-24-02-24-L,2024-02-25,2024-02-26,UDMS,2021-03-12
8,1610001861711,7,S2A,20L2161350,LEYLAND METERING SERVICES LTD,MRCY,2733.6,E-2R-VAR-22-11-01-G,2024-02-15,2024-02-16,UDMS,2024-01-04
9,2200023066430,11,S2ADE,21L3274003,OCTOPUS ENERGY SERVICES LTD,MRCY,3583.1,E-2R-VAR-22-11-01-L,2024-02-21,2024-02-22,UDMS,2023-05-11


In [3]:
# date and time for titles, headers and footers
import os
from datetime import datetime as dt, timedelta
now = dt.now()
one_year_ago = now - timedelta(days=365)
now_1 = now - timedelta(days=1)
now_2 = now - timedelta(days=2)
now_3 = now - timedelta(days=3)
now_4 = now - timedelta(days=4)
now_5 = now - timedelta(days=5)
formatted_dt = now.strftime('%Y%m%d%H%M%S')
file_id_dt = now.strftime('%y%m%d')
now_minus_1_year = one_year_ago.strftime('%Y%m%d')
print(one_year_ago)
file_id_dt_delay1 = now_1.strftime('%y%m%d')
file_id_dt_delay2 = now_2.strftime('%y%m%d')
file_id_dt_delay3 = now_3.strftime('%y%m%d')
file_id_dt_delay4 = now_4.strftime('%y%m%d')
file_id_dt_delay5 = now_5.strftime('%y%m%d')

print("^ Date + Time")

2023-02-26 17:15:10.819085
^ Date + Time


In [4]:
# ***** functions for writing d52 / d205 / d367 / cmex *****
print("^ Functions")

def generate_onerate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC, d52_body_output, d52_body_counter):
    onerate_d52_content = f'''121|{MPAN}|{formatted_SSD}|1|{formatted_EFD}|A|{formatted_EFD}|\n122|{SSC}|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00001|{EAC}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(onerate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_tworate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter):
    tworate_d52_content = f'''121|{MPAN}|{formatted_SSD}|2|{formatted_EFD}|A|{formatted_EFD}|\n122|{SSC}|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00043|{EAC_TPR_1}|{formatted_EFD}|\n124|00210|{EAC_TPR_2}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(tworate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_GSP_N_tworate_d52_content(MPAN, formatted_EFD, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter):
    tworate_d52_content = f'''121|{MPAN}|{formatted_SSD}|2|{formatted_EFD}|A|{formatted_EFD}|\n122|0722|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|13068|{EAC_TPR_1}|{formatted_EFD}|\n124|13067|{EAC_TPR_2}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(tworate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1


def generate_GSP_P_onerate_d52_content(MPAN, formatted_EFD, GSP, EAC, d52_body_output, d52_body_counter):
    onerate_d52_content = f'''121|{MPAN}|{formatted_SSD}|1|{formatted_EFD}|A|{formatted_EFD}|\n122|0801|{formatted_EFD}|_{GSP}|{formatted_EFD}|\n124|00306|{EAC}|{formatted_EFD}|'''

    with open(d52_body_output, 'a') as body:
        body.write(onerate_d52_content)
        body.write('\n')

    d52_body_counter[0] += 1



def generate_d205_content(MPAN, d205_formatted_EFD, SSC, MTC, PC, d205_body_ouput, d205_body_counter):
    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{d205_formatted_EFD}|||||{MTC}|{d205_formatted_EFD}|{PC}|{d205_formatted_EFD}|{SSC}|{d205_formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_GSP_P_d205_content(MPAN, formatted_EFD, d205_body_ouput, d205_body_counter):

    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{d205_formatted_EFD}|||||801|{d205_formatted_EFD}|1|{d205_formatted_EFD}|0801|{d205_formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_GSP_N_d205_content(MPAN, formatted_EFD, d205_body_ouput, d205_body_counter):
    d205_content = f'''452|{d205_body_counter[0]}|SP02|{MPAN}|{formatted_SSD}|||||005|{d205_formatted_EFD}|2|{d205_formatted_EFD}|0722|{d205_formatted_EFD}|||||||||||||||'''

    with open(d205_body_output, 'a') as body:
        body.write(d205_content)
        body.write('\n')

    d205_body_counter [0] += 1


def generate_onerate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    onerate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|{SSC}|{formatted_EFD}|\n03J|00001|\n04J|{MSN}|\n05J|1|Standard|1|C|AI|1.00|5|'''

    with open (d367_body_output, 'a') as body:
        body.write(onerate_d367_content)
        body.write('\n')

    d367_body_counter [0] += 1


def generate_tworate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    tworate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|{SSC}|{formatted_EFD}|\n03J|00043|\n04J|{MSN}|\n05J|1|Day|1|C|AI|1.00|5|\n03J|00210|\n04J|{MSN}|\n05J|2|Night|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(tworate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_tworate_GSP_N_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter):
    tworate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|0722|{formatted_EFD}|\n03J|13068|\n04J|{MSN}|\n05J|1|Day|1|C|AI|1.00|5|\n03J|13067|\n04J|{MSN}|\n05J|2|Night|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(tworate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_onerate_GSP_P_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter):
    onerate_d367_content = f'''01J|{MPAN}|{formatted_EFD}|R|{formatted_EFD}|{TYPE}|\n02J|0801|{formatted_EFD}|\n03J|00001|\n04J|{MSN}|\n05J|1|Standard|1|C|AI|1.00|5|'''

    with open(d367_body_output, 'a') as body:
        body.write(onerate_d367_content)
        body.write('\n')

    d367_body_counter[0] += 1


def generate_onerate_cmex_content(MPAN, MSN, EFD, INITIAL1, INITIAL2, FINAL1):
    onerate_cmex_content = (f'''
            MPAN :: {MPAN} 
            MSN :: {MSN}
            Date of CMEX :: {EFD} 
            Initial Reads :: 1:{INITIAL1}   2:{INITIAL2}
            Final Read :: {FINAL1}''')
    return onerate_cmex_content


def generate_tworate_cmex_content(MPAN, MSN, EFD, INITIAL1, FINAL1, FINAL2):
    tworate_cmex_content = (f'''
            MPAN :: {MPAN}
            MSN :: {MSN}
            Date of CMEX :: {EFD}
            Initial Read :: 1:{INITIAL1}
            Final Read :: 1:{FINAL1}    2:{FINAL2}''')
    return tworate_cmex_content


def d52_headfoot_output(d52_filename, code, d52_seq_number, formatted_dt, SUPPLIER, DC, d52_counter):
    with open (d52_filename, 'r') as count_d52:
        d52_lines = count_d52.readlines()
    
    total_d52_lines = len(d52_lines)
    
    with open(d52_filename, 'r') as d52_body:
        d52_existing_content = d52_body.read()
        
    d52_header = (f"ZHV|{d52_seq_number}{code}|D0052001|X|{SUPPLIER}|D|{DC}|{formatted_dt}||||OPER|")
    d52_footer = (f"ZPT|{d52_seq_number}{code}|{total_d52_lines}||{d52_counter[0]}|{formatted_dt}|")
    
    with open(d52_filename, 'w') as d52_headfoot:
        d52_headfoot.write(d52_header)
        d52_headfoot.write('\n')
        d52_headfoot.write(d52_existing_content)
        d52_headfoot.write(d52_footer)
        d52_headfoot.write('\n')
        

def d205_headfoot_output(d205_filename, d205_seq_number, DNO, formatted_dt, chosen_d205_counter, SUPPLIER):
    with open(d205_filename, 'r') as count_d205:
        d205_lines = count_d205.readlines()

    total_d205_lines = len(d205_lines)
    # add 1 for |752| line which won't figure in the counter of MPAN executions
    total_d205_lines += 1

    with open(d205_filename, 'r') as d205_body:
        d205_existing_content = d205_body.read()
        
    with open(d205_752_seq, 'r') as count_752:
        d205_seq_header = count_752.read()
        d205_seq_header = int(d205_seq_header)
        d205_seq_header += 1
        d205_seq_header = str(d205_seq_header)
        
    with open(d205_752_seq, 'w') as write_752:
        write_752.write(d205_seq_header)

    d205_header = f'ZHV|{d205_seq_number}|D0205001|X|{SUPPLIER}|P|{DNO}|{formatted_dt}||||OPER|\n752|{d205_seq_header}|'
    d205_footer = f'ZPT|{d205_seq_number}|{total_d205_lines}||{chosen_d205_counter[0]}|{formatted_dt}|'

    with open(d205_filename, 'w') as d205_headfoot:
        d205_headfoot.write(d205_header)
        d205_headfoot.write('\n')
        d205_headfoot.write(d205_existing_content)
        d205_headfoot.write(d205_footer)
        

def d367_headfoot_output(d367_filename, d367_seq_number, SUPPLIER, MOP, formatted_dt, d367_body_counter):
    with open(d367_filename, 'r') as count_d367:
        d367_lines = count_d367.readlines()
    
    total_d367_lines = len(d367_lines)
    
    with open(d367_filename, 'r') as d367_body:
        d367_existing_content = d367_body.read()
        
    d367_header = (f"ZHV|{d367_seq_number}|D0367001|X|{SUPPLIER}|M|{MOP}|{formatted_dt}||||OPER|")
    d367_footer = (f"ZPT|{d367_seq_number}|{total_d367_lines}||{d367_body_counter}|{formatted_dt}|")

    with open(d367_filename, 'w') as d367_headfoot:
        d367_headfoot.write(d367_header)
        d367_headfoot.write('\n')
        d367_headfoot.write(d367_existing_content)
        d367_headfoot.write(d367_footer)


def delete_pre_script (directory):
    if os.path.exists(directory) and os.path.isdir(directory):
        outputs = os.listdir(directory)

        for file in outputs:
            file_path = os.path.join(directory, file)

            if os.path.isfile(file_path):
                os.remove(file_path)


def delete_empty_files(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and os.path.getsize(filepath) == 0:
            os.remove(filepath)

^ Functions


In [5]:
# ***** file directories *****
print("^ File Directories + Containers + Counters")

# containers
flows = []
d52s = []
d205s = []
d367s = []

flows.append(d52s)
flows.append(d205s)
flows.append(d367s)


# csv + seq number pathways
configuration_csv = '/home/jovyan/shared/Settlement/SSC Changer/variables.csv'
d205_752_seq = '/home/jovyan/shared/Settlement/SSC Changer/752.txt'

# delete previous flow outputs
delete_d10s = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/'
delete_52_367 = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/'
delete_d205s = f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/'
delete_pre_script(directory=delete_52_367)
delete_pre_script(directory=delete_d205s)


# ***** variables for flow header + footer *****
# counters for instances section of footers
mrcy_udms_counter = [0]; mrcy_accu_counter = [0]; mrcy_d205_EELC_counter = [0]; mrcy_d205_EMEB_counter = [0];
mrcy_d205_LOND_counter = [0]; mrcy_d205_MANW_counter = [0]; mrcy_d205_MIDE_counter = [0]; mrcy_d205_NEEB_counter = [0]; mrcy_d205_NORW_counter = [0];
mrcy_d205_HYDE_counter = [0]; mrcy_d205_SPOW_counter = [0]; mrcy_d205_SEEB_counter = [0]; mrcy_d205_SOUT_counter = [0]; mrcy_d205_SWAE_counter = [0];
mrcy_d205_SWEB_counter = [0]; mrcy_d205_YELG_counter = [0]; mrcy_d205_IPNL_counter = [0]; mrcy_d205_LENG_counter = [0]; mrcy_d205_GUCL_counter = [0];
mrcy_d205_ETCL_counter = [0]; mrcy_d205_EDFI_counter = [0]; mrcy_d205_HARL_counter = [0]; mrcy_d205_PENL_counter = [0]; mrcy_d205_UKPD_counter = [0];
mrcy_d205_UDNL_counter = [0]; mrcy_d205_GGEN_counter = [0]; mrcy_d205_MPDL_counter = [0]; mrcy_d205_FEAL_counter = [0]; mrcy_d205_VATT_counter = [0];
mrcy_d205_FORB_counter = [0]; mrcy_d205_INDI_counter = [0]; mrcy_d367_upld_counter = [0]; mrcy_d367_aesm_counter = [0]; mrcy_d367_lmsl_counter = [0]; 
mrcy_d367_siem_counter = [0]; mrcy_d367_bmsl_counter = [0]; mrcy_d367_ukdc_counter = [0]; rege_accu_counter = [0]; rege_d205_EELC_counter = [0];
rege_d205_LOND_counter = [0]; rege_d205_MANW_counter = [0]; rege_d205_MIDE_counter = [0]; rege_d205_NEEB_counter = [0]; rege_d205_NORW_counter = [0];
rege_d205_HYDE_counter = [0]; rege_d205_SPOW_counter = [0]; rege_d205_SEEB_counter = [0]; rege_d205_SOUT_counter = [0]; rege_d205_SWAE_counter = [0];
rege_d205_SWEB_counter = [0]; rege_d205_YELG_counter = [0]; rege_d205_IPNL_counter = [0]; rege_d205_LENG_counter = [0]; rege_d205_GUCL_counter = [0];
rege_d205_ETCL_counter = [0]; rege_d205_EDFI_counter = [0]; rege_d205_HARL_counter = [0]; rege_d205_PENL_counter = [0]; rege_d205_UKPD_counter = [0];
rege_d205_UDNL_counter = [0]; rege_d205_GGEN_counter = [0]; rege_d205_MPDL_counter = [0]; rege_d205_FEAL_counter = [0]; rege_d205_VATT_counter = [0];
rege_d205_FORB_counter = [0]; rege_d205_INDI_counter = [0]; rege_d367_upld_counter = [0]; rege_d367_aesm_counter = [0]; rege_d367_lmsl_counter = [0]; 
rege_d367_siem_counter = [0]; rege_d367_bmsl_counter = [0]; rege_d367_ukdc_counter = [0]; rege_d205_EMEB_counter = [0]; rege_udms_counter = [0];



^ File Directories + Containers + Counters


In [6]:
# dictionaries
print("^ Dictionaries")

# ***** d52 output dictionary *****
d52_dictionary = {
    'MRCY_UDMS': {'d52_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_UDMS_GR{file_id_dt}0b.D0052', 'd52_body_counter': mrcy_udms_counter},
    'MRCY_ACCU': {'d52_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_ACCU_GR{file_id_dt}1b.D0052', 'd52_body_counter': mrcy_accu_counter},
    'REGE_UDMS': {'d52_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_UDMS_GR{file_id_dt}3b.D0052', 'd52_body_counter': rege_udms_counter},
    'REGE_ACCU': {'d52_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_ACCU_GR{file_id_dt}4b.D0052', 'd52_body_counter': rege_accu_counter},
}

# ***** d205 output dictionary (DNOs + iDNOs) *****
d205_dictionary = {
    'MRCY_10': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_EELC_GR{file_id_dt}0c.D0205', 'd205_body_counter': mrcy_d205_EELC_counter},
    'MRCY_11': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_EMEB_GR{file_id_dt}1c.D0205', 'd205_body_counter': mrcy_d205_EMEB_counter},
    'MRCY_12': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_LOND_GR{file_id_dt}2c.D0205', 'd205_body_counter': mrcy_d205_LOND_counter},
    'MRCY_13': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_MANW_GR{file_id_dt}3c.D0205', 'd205_body_counter': mrcy_d205_MANW_counter},
    'MRCY_14': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_MIDE_GR{file_id_dt}4c.D0205', 'd205_body_counter': mrcy_d205_MIDE_counter},
    'MRCY_15': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_NEEB_GR{file_id_dt}5c.D0205', 'd205_body_counter': mrcy_d205_NEEB_counter},
    'MRCY_16': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_NORW_GR{file_id_dt}6c.D0205', 'd205_body_counter': mrcy_d205_NORW_counter},
    'MRCY_17': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_HYDE_GR{file_id_dt}7c.D0205', 'd205_body_counter': mrcy_d205_HYDE_counter},
    'MRCY_18': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SPOW_GR{file_id_dt}8c.D0205', 'd205_body_counter': mrcy_d205_SPOW_counter},
    'MRCY_19': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SEEB_GR{file_id_dt}9c.D0205', 'd205_body_counter': mrcy_d205_SEEB_counter},
    'MRCY_20': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SOUT_GR{file_id_dt_delay1}0c.D0205', 'd205_body_counter': mrcy_d205_SOUT_counter},
    'MRCY_21': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SWAE_GR{file_id_dt_delay1}1c.D0205', 'd205_body_counter': mrcy_d205_SWAE_counter},
    'MRCY_22': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SWEB_GR{file_id_dt_delay1}2c.D0205', 'd205_body_counter': mrcy_d205_SWEB_counter},
    'MRCY_23': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_YELG_GR{file_id_dt_delay1}3c.D0205', 'd205_body_counter': mrcy_d205_YELG_counter},
    'MRCY_24': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_IPNL_GR{file_id_dt_delay2}8c.D0205', 'd205_body_counter': mrcy_d205_IPNL_counter},
    'MRCY_25': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_LENG_GR{file_id_dt_delay2}9c.D0205', 'd205_body_counter': mrcy_d205_LENG_counter},
    'MRCY_26': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_GUCL_GR{file_id_dt_delay3}0c.D0205', 'd205_body_counter': mrcy_d205_GUCL_counter},
    'MRCY_27': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_ETCL_GR{file_id_dt_delay3}1c.D0205', 'd205_body_counter': mrcy_d205_ETCL_counter},
    'MRCY_28': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_EDFI_GR{file_id_dt_delay3}2c.D0205', 'd205_body_counter': mrcy_d205_EDFI_counter},
    'MRCY_29': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_HARL_GR{file_id_dt_delay3}3c.D0205', 'd205_body_counter': mrcy_d205_HARL_counter},
    'MRCY_30': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_PENL_GR{file_id_dt_delay3}4c.D0205', 'd205_body_counter': mrcy_d205_PENL_counter},
    'MRCY_31': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_UKPD_GR{file_id_dt_delay3}5c.D0205', 'd205_body_counter': mrcy_d205_UKPD_counter},
    'MRCY_32': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_UDNL_GR{file_id_dt_delay3}6c.D0205', 'd205_body_counter': mrcy_d205_UDNL_counter},
    'MRCY_33': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_GGEN_GR{file_id_dt_delay3}7c.D0205', 'd205_body_counter': mrcy_d205_GGEN_counter},
    'MRCY_34': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_MPDL_GR{file_id_dt_delay3}8c.D0205', 'd205_body_counter': mrcy_d205_MPDL_counter},
    'MRCY_35': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_FEAL_GR{file_id_dt_delay3}9c.D0205', 'd205_body_counter': mrcy_d205_FEAL_counter},
    'MRCY_36': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_VATT_GR{file_id_dt_delay4}0c.D0205', 'd205_body_counter': mrcy_d205_VATT_counter},
    'MRCY_37': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_FORB_GR{file_id_dt_delay4}1c.D0205', 'd205_body_counter': mrcy_d205_FORB_counter},
    'MRCY_38': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_INDI_GR{file_id_dt_delay4}2c.D0205', 'd205_body_counter': mrcy_d205_INDI_counter},
    'REGE_10': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_EELC_GR{file_id_dt}0c.D0205', 'd205_body_counter': rege_d205_EELC_counter},
    'REGE_11': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_EMEB_GR{file_id_dt}1c.D0205', 'd205_body_counter': rege_d205_EMEB_counter},
    'REGE_12': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_LOND_GR{file_id_dt}2c.D0205', 'd205_body_counter': rege_d205_LOND_counter},
    'REGE_13': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_MANW_GR{file_id_dt}3c.D0205', 'd205_body_counter': rege_d205_MANW_counter},
    'REGE_14': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_MIDE_GR{file_id_dt}4c.D0205', 'd205_body_counter': rege_d205_MIDE_counter},
    'REGE_15': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_NEEB_GR{file_id_dt}5c.D0205', 'd205_body_counter': rege_d205_NEEB_counter},
    'REGE_16': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_NORW_GR{file_id_dt}6c.D0205', 'd205_body_counter': rege_d205_NORW_counter},
    'REGE_17': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_HYDE_GR{file_id_dt}7c.D0205', 'd205_body_counter': rege_d205_HYDE_counter},
    'REGE_18': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SPOW_GR{file_id_dt}8c.D0205', 'd205_body_counter': rege_d205_SPOW_counter},
    'REGE_19': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SEEB_GR{file_id_dt}9c.D0205', 'd205_body_counter': rege_d205_SEEB_counter},
    'REGE_20': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SOUT_GR{file_id_dt_delay1}0c.D0205', 'd205_body_counter': rege_d205_SOUT_counter},
    'REGE_21': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SWAE_GR{file_id_dt_delay1}1c.D0205', 'd205_body_counter': rege_d205_SWAE_counter},
    'REGE_22': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SWEB_GR{file_id_dt_delay1}2c.D0205', 'd205_body_counter': rege_d205_SWEB_counter},
    'REGE_23': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_YELG_GR{file_id_dt_delay1}3c.D0205', 'd205_body_counter': rege_d205_YELG_counter},
    'REGE_24': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_IPNL_GR{file_id_dt_delay2}8c.D0205', 'd205_body_counter': rege_d205_IPNL_counter},
    'REGE_25': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_LENG_GR{file_id_dt_delay2}9c.D0205', 'd205_body_counter': rege_d205_LENG_counter},
    'REGE_26': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_GUCL_GR{file_id_dt_delay3}0c.D0205', 'd205_body_counter': rege_d205_GUCL_counter},
    'REGE_27': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_ETCL_GR{file_id_dt_delay3}1c.D0205', 'd205_body_counter': rege_d205_ETCL_counter},
    'REGE_28': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_EDFI_GR{file_id_dt_delay3}2c.D0205', 'd205_body_counter': rege_d205_EDFI_counter},
    'REGE_29': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_HARL_GR{file_id_dt_delay3}3c.D0205', 'd205_body_counter': rege_d205_HARL_counter},
    'REGE_30': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_PENL_GR{file_id_dt_delay3}4c.D0205', 'd205_body_counter': rege_d205_PENL_counter},
    'REGE_31': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_UKPD_GR{file_id_dt_delay3}5c.D0205', 'd205_body_counter': rege_d205_UKPD_counter},
    'REGE_32': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_UDNL_GR{file_id_dt_delay3}6c.D0205', 'd205_body_counter': rege_d205_UDNL_counter},
    'REGE_33': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_GGEN_GR{file_id_dt_delay3}7c.D0205', 'd205_body_counter': rege_d205_GGEN_counter},
    'REGE_34': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_MPDL_GR{file_id_dt_delay3}8c.D0205', 'd205_body_counter': rege_d205_MPDL_counter},
    'REGE_35': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_FEAL_GR{file_id_dt_delay3}9c.D0205', 'd205_body_counter': rege_d205_FEAL_counter},
    'REGE_36': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_VATT_GR{file_id_dt_delay4}0c.D0205', 'd205_body_counter': rege_d205_VATT_counter},
    'REGE_37': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_FORB_GR{file_id_dt_delay4}1c.D0205', 'd205_body_counter': rege_d205_FORB_counter},
    'REGE_38': {'d205_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_INDI_GR{file_id_dt_delay4}2c.D0205', 'd205_body_counter': rege_d205_INDI_counter},
}

d367_dictionary = {
    'MRCY_AESM': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_AESM_GR{file_id_dt}0g.D0367', 'd367_body_counter': mrcy_d367_aesm_counter},
    'MRCY_LMSL': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_LMSL_GR{file_id_dt}1g.D0367', 'd367_body_counter': mrcy_d367_lmsl_counter},
    'MRCY_UPLD': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_UPLD_GR{file_id_dt}2g.D0367', 'd367_body_counter': mrcy_d367_upld_counter},
    'MRCY_SIEM': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_SIEM_GR{file_id_dt}3g.D0367', 'd367_body_counter': mrcy_d367_siem_counter},
    'MRCY_BMSL': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_BMSL_GR{file_id_dt}4g.D0367', 'd367_body_counter': mrcy_d367_bmsl_counter},
    'MRCY_UKDC': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/MRCY_UKDC_GR{file_id_dt}5g.D0367', 'd367_body_counter': mrcy_d367_ukdc_counter},
    'REGE_AESM': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_UPLD_GR{file_id_dt}0g.D0367', 'd367_body_counter': rege_d367_aesm_counter},
    'REGE_LMSL': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_AESM_GR{file_id_dt}1g.D0367', 'd367_body_counter': rege_d367_lmsl_counter},
    'REGE_UPLD': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_LMSL_GR{file_id_dt}2g.D0367', 'd367_body_counter': rege_d367_upld_counter},
    'REGE_SIEM': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_SIEM_GR{file_id_dt}3g.D0367', 'd367_body_counter': rege_d367_siem_counter},
    'REGE_BMSL': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_BMSL_GR{file_id_dt}4g.D0367', 'd367_body_counter': rege_d367_bmsl_counter},
    'REGE_UKDC': {'d367_body_output': f'/home/jovyan/shared/Settlement/SSC Changer/Outputs/REGE_UKDC_GR{file_id_dt}5g.D0367', 'd367_body_counter': rege_d367_ukdc_counter},
}

^ Dictionaries


In [7]:
# ***** import data and define variables *****
print("^ Parse CSV + Define Variables + Write Flows")

# csv import
import csv
from datetime import datetime

with open(configuration_csv, 'r') as infile:
    reader = csv.reader(infile, delimiter=',')
    header = next(reader)

    for row in reader:
        MPAN = row[1]
        mpan_prefix = MPAN[0:2]
        GSP = row[2]
        TYPE = row[3]
        MSN = row[4]
        MOP = row[5]
        SUPPLIER = row[6]
        EAC = row[7]
        EFD = row[10]
        TARIFF = row[8]
        DC = row[11]
        SSC = '9999'
        MTC = 000
        PC = 0
        DNO = row[2]
        SSD = row[12]
        # Error handling for if EAC is 'none'
        try:
            EAC = float(EAC)
        except(ValueError):
            EAC = input(f"""No EAC found for {MPAN}, please enter the EAC! ***Numbers only, no decimals***. \n
            If the meter is two rate presently, add the EACs together""")
            try:
                EAC = float(EAC)
            except(ValueError):
                print(f"Non numerical characters entered, skipping {MPAN}")
        # Error handling for date/time formatting as nulls pulled through SQL in place of dates can cause ValueError
        try:
            EFD_datetime = datetime.strptime(EFD, '%Y-%m-%d')
            formatted_EFD = EFD_datetime.strftime('%Y%m%d')
            SSD_datetime = datetime.strptime(SSD, '%Y-%m-%d')
            formatted_SSD = SSD_datetime.strftime('%Y%m%d')
            
            # Still work to be done here >> Trying to set D205s to adjust EFD if its >1 year ago but d205_EFD_datetime = 
            d205_EFD_datetime = datetime.strptime(EFD, '%Y-%m-%d')
            d205_formatted_EFD = EFD_datetime.strftime('%Y%m%d')
            print(d205_formatted_EFD, now_minus_1_year)
            if d205_formatted_EFD < now_minus_1_year:
                d205_EFD_datetime = now_minus_1_year
                #print(now_minus_1_year)
                #print(d205_formatted_EFD)
        except(ValueError):
            print(f"Date Error for {MPAN}")
            EFD = input(f"Would you like to enter a manual EFD (yyyy-mm-dd) for {MPAN}? Press n to break")
            if EFD == 'n':
                continue
            else: 
                try: 
                    EFD_datetime = datetime.strptime(EFD, '%Y-%m-%d')
                    formatted_EFD = EFD_datetime.strftime('%Y%m%d')
                    SSD_datetime = datetime.strptime(SSD, '%Y-%m-%d')
                    formatted_SSD = SSD_datetime.strftime('%Y%m%d')
                    
                    d205_EFD_datetime = datetime.strptime(EFD, '%Y-%m-%d')
                    d205_formatted_EFD = EFD_datetime.strftime('%Y%m%d')
                    print(d205_formatted_EFD, now_minus_1_year)
                    if d205_formatted_EFD < now_minus_1_year:
                        d205_EFD_datetime = now_minus_1_year
                        print(now_minus_1_year)
                        print(d205_formatted_EFD)
                except(ValueError):
                    print("Date not formatted correctly, skipping and adding to error logs {MPAN}")
                    print("\n\n")
                  

        # format desired SSC
        if "1R" in TARIFF:
            SSC = '0393'
        if "2R" in TARIFF :
            SSC = '0151'
        
        # conditional formatting for MTC
        if SSC == '0393':
            PC = 1
            MTC = 801

        if SSC == '0151':
            PC = 2
            MTC = 811

        # Conditional formatting for GSPs, DNOs defined in D205 arguments -
        if GSP == '1':
            GSP = 'A'
        if GSP == '2':
            GSP = 'B'
        if GSP == '3':
            GSP = 'C'
        if GSP == '4':
            GSP = 'D'
        if GSP == '5':
            GSP = 'E'
        if GSP == '6':
            GSP = 'F'
        if GSP == '7':
            GSP = 'G'
        if GSP == '8':
            GSP = 'H'
        if GSP == '9':
            GSP = 'J'
        if GSP == '10':
            GSP = 'K'
        if GSP == '11':
            GSP = 'L'
        if GSP == '12':
            GSP = 'M'
        if GSP == '13':
            GSP = 'N'
        if GSP == '14':
            GSP = 'P'

        # Conditional formatting for Northern Scotland 0801
        if GSP == '14':
            SSC = '0801'
        if GSP == '13':
            SSC = '0722'
            
        # MOP MPID Formatting
        if MOP == 'AES Smart Metering Ltd':
            MOP = 'AESM'
        if MOP == 'LEYLAND METERING SERVICES LTD':
            MOP = 'LMSL'
        if MOP == 'OCTOPUS ENERGY SERVICES LTD':
            MOP = 'LMSL'
        if MOP == 'SMS Energy Services Ltd':
            MOP = 'UPLD'
        if MOP == 'Siemens MCS':
            MOP = 'SIEM'
        if MOP == 'BURY METERING SERVICES LTD':
            MOP = 'BMSL'
        if MOP == 'IMServ Europe Ltd':
            MOP = 'UKDC'
        
        
                    
        # ***** write d52 / d205 / d367 / cmex *****
        # write d52 + EAC Edit
        if SSC == '0151':
            EAC_TPR_1 = round(EAC * 0.667, 1)
            EAC_TPR_2 = round(EAC * 0.333, 1)
        
        d52_key = f'{SUPPLIER}_{DC}'

        if d52_key in d52_dictionary:
            d52_dict_output = d52_dictionary[d52_key]
        d52_body_output = d52_dict_output['d52_body_output']
        d52_body_counter = d52_dict_output['d52_body_counter']
        
        if d52_body_output not in d52s:
                d52s.append(d52_body_output)
                
        if GSP == 'P' and SSC == '0393':
            generate_GSP_P_onerate_d52_content(MPAN, formatted_EFD, GSP, EAC, d52_body_output, d52_body_counter)

        if GSP != 'P' and SSC == '0393':
            generate_onerate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC, d52_body_output, d52_body_counter)
            
        if GSP == 'N' and SSC == '0151':
            generate_GSP_N_tworate_d52_content(MPAN, formatted_EFD, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter)

        if GSP !='N' and SSC == '0151':
            generate_tworate_d52_content(MPAN, formatted_EFD, SSC, GSP, EAC_TPR_1, EAC_TPR_2, d52_body_output, d52_body_counter)

                
        # write d205
        d205_key = f'{SUPPLIER}_{mpan_prefix}'

        if d205_key in d205_dictionary:
            d205_dict_output = d205_dictionary[d205_key]
        d205_body_output = d205_dict_output['d205_body_output']
        d205_body_counter = d205_dict_output['d205_body_counter']
        
        if d205_body_output not in d205s:
            d205s.append(d205_body_output)

        if GSP == 'P' and SSC == '0393':
            generate_GSP_P_d205_content(MPAN, d205_formatted_EFD, d205_body_ouput, d205_body_counter)
            
        if GSP == 'P' and SSC == '0151':
            generate_d205_content(MPAN, d205_formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)
 
        if GSP == 'N' and SSC == '0151':
            generate_GSP_N_d205_content(MPAN, d205_formatted_EFD, d205_body_output, d205_body_counter)
            
        if GSP == 'N' and SSC == '0393':
            generate_d205_content(MPAN, d205_formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)

        if GSP != 'P' and GSP != 'N':
            generate_d205_content(MPAN, d205_formatted_EFD, SSC, MTC, PC, d205_body_output, d205_body_counter)
            

        # write d367
        d367_key = f'{SUPPLIER}_{MOP}'

        if d367_key in d367_dictionary:
            d367_dict_output = d367_dictionary[d367_key]
        d367_body_output = d367_dict_output['d367_body_output']
        d367_body_counter = d367_dict_output['d367_body_counter']
        
        if d367_body_output not in d367s:
            d367s.append(d367_body_output)

        if GSP == 'N' and SSC == '0151':
            generate_tworate_GSP_N_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter)

        if GSP !='N' and SSC == '0151':
            generate_tworate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter)

        if SSC == '0393' and GSP == 'P':
            generate_onerate_GSP_P_d367_content(MPAN, TYPE, MSN, formatted_EFD, d367_body_output, d367_body_counter)

        if GSP !='P' and SSC == '0393':
            generate_onerate_d367_content(MPAN, TYPE, MSN, formatted_EFD, SSC, d367_body_output, d367_body_counter)
            

^ Parse CSV + Define Variables + Write Flows
20240215 20230226
MRCY UDMS
20240223 20230226
MRCY UDMS
20240221 20230226
MRCY UDMS
20240224 20230226
MRCY UDMS
20240222 20230226
MRCY UDMS
20240227 20230226
MRCY UDMS
20240224 20230226
MRCY UDMS
20240226 20230226
MRCY UDMS
20240216 20230226
MRCY UDMS
20240222 20230226
MRCY UDMS
20240221 20230226
MRCY UDMS
20240215 20230226
MRCY UDMS
Date Error for 1417489971007


Would you like to enter a manual EFD (yyyy-mm-dd) for 1417489971007? Press n to break 2024-01-01


20240101 20230226
MRCY UDMS
Date Error for 2000052732818


Would you like to enter a manual EFD (yyyy-mm-dd) for 2000052732818? Press n to break 2024-01-01


20240101 20230226
MRCY UDMS


In [8]:
# ***** write header/footers and delete unused files *****
print("^ Write Headers/Footers + Delete Unused Files")

# d52 header/footer
for d52 in d52s:
    code = d52[69:71]; SUPPLIER = d52[51:55]; DC = d52[56:60]; d52_seq_number = d52[61:69]; supplier_dc = f"{SUPPLIER}_{DC}".lower(); counter = globals()[f'{supplier_dc}_counter']
    d52_headfoot_output(d52_filename=f'{d52}', code=code, d52_seq_number = d52_seq_number, formatted_dt=formatted_dt, SUPPLIER=SUPPLIER, d52_counter=counter, DC=DC)

# d205 header/footer
for d205 in d205s:
    SUPPLIER = d205[51:55]; supplier = SUPPLIER.lower(); DNO = d205[56:60]; seq_number = d205[61:71]; counter = globals()[f'{supplier}_d205_{DNO}_counter']
    d205_headfoot_output(d205_filename=f'{d205}', chosen_d205_counter=counter, DNO=f'{DNO}', d205_seq_number=f'{seq_number}', formatted_dt=formatted_dt, SUPPLIER=f'{SUPPLIER}')

# d367 header/footer    
for d367 in d367s:
    SUPPLIER = d367[51:55]; MOP = d367[56:60]; seq_number = d367[61:71]; supplier_mop = f"{SUPPLIER}_d367_{MOP}".lower(); counter = globals()[f'{supplier_mop}_counter']
    d367_headfoot_output(d367_filename=f'{d367}', d367_seq_number=f'{seq_number}', SUPPLIER=f'{SUPPLIER}', MOP=f'{MOP}', formatted_dt=formatted_dt, d367_body_counter=counter[0])

# delete empty files
directory_path_main = '/home/jovyan/shared/Settlement/SSC Changer/Outputs/'
delete_empty_files(directory_path_main)


^ Write Headers/Footers + Delete Unused Files
